In [1]:
# Using pysal-workshop env
import json, os
import pandas as pd
import geopandas as gpd
from here_location_services import LS
from here_location_services.config.routing_config import ROUTING_RETURN

## Geocoding

In [2]:
sites_df = pd.read_json("../data/covid_sites.json")

In [3]:
sites_df.head()

,site_name,address,suburb,postcode,period,note,date_added,advice,tier,full_address
0,Dukes Gym Abbotsford,571-573 Victoria Street,Abbotsford,3067,24/05/2021 6:15pm - 8:15pm,Case attended venue,26/05/2021,Tier 1 - Get tested immediately and quarantin...,Tier 1,"571-573 Victoria Street , Abbotsford , Victo..."
1,Chelsea Heights Hotel,Springvale Road & Wells Road,Aspendale Gardens,3196,23/05/2021 12:10pm - 1:30pm,Case dined at venue,27/05/2021,Tier 1 - Get tested immediately and quarantin...,Tier 1,"Springvale Road & Wells Road , Aspendale Gar..."
2,Axedale Tavern,105 High Street,Axedale,3551,23/05/2021 11:45am - 1:30pm,Case attended function,26/05/2021,Tier 1 - Get tested immediately and quarantin...,Tier 1,"105 High Street , Axedale , Victoria, 3551 ..."
3,Endless Lifestyle Studio\r\n\r,Shop 3/188 Whitehorse Road\r\n\r,Balwyn,3103,23/05/2021 9:45am - 12:40pm,Case attended venue,26/05/2021,Tier 1 - Get tested immediately and quarantin...,Tier 1,"Shop 3/188 Whitehorse Road\r\n\r , Balwyn , ..."
4,Mcdonalds Bayswater North,296 Canterbury Road,Bayswater North,3153,21/05/2021 11:00pm - 12:00am,Case dined in restaurant,26/05/2021,Tier 1 - Get tested immediately and quarantin...,Tier 1,"296 Canterbury Road , Bayswater North , Vict..."


In [4]:
LS_API_KEY = os.environ.get("HERE_API_KEY")  # Get API KEY from environment.
ls = LS(api_key=LS_API_KEY)

In [5]:
def geo_coding(address):
    geo = ls.geocode(query=address)
    position = geo.response['items'][0]['position']
    return [position['lat'], position['lng']]

In [6]:
# Test
address = "Shop TG-01, Uni Hill Town Centre, 1-3 Janefield Drive ,  Bundoora , Victoria, 3083, Australia"
geo_coding(address)

[-37.68114, 145.07049]

In [7]:
%%time
sites_df['location'] = sites_df.full_address.apply(lambda x: geo_coding(x))

CPU times: user 3.95 s, sys: 70.1 ms, total: 4.02 s
Wall time: 1min 35s


In [8]:
sites_df.shape

(158, 11)

In [9]:
sites_df.head()

,site_name,address,suburb,postcode,period,note,date_added,advice,tier,full_address,location
0,Dukes Gym Abbotsford,571-573 Victoria Street,Abbotsford,3067,24/05/2021 6:15pm - 8:15pm,Case attended venue,26/05/2021,Tier 1 - Get tested immediately and quarantin...,Tier 1,"571-573 Victoria Street , Abbotsford , Victo...","[-37.81089, 145.00567]"
1,Chelsea Heights Hotel,Springvale Road & Wells Road,Aspendale Gardens,3196,23/05/2021 12:10pm - 1:30pm,Case dined at venue,27/05/2021,Tier 1 - Get tested immediately and quarantin...,Tier 1,"Springvale Road & Wells Road , Aspendale Gar...","[-38.02461, 145.1341]"
2,Axedale Tavern,105 High Street,Axedale,3551,23/05/2021 11:45am - 1:30pm,Case attended function,26/05/2021,Tier 1 - Get tested immediately and quarantin...,Tier 1,"105 High Street , Axedale , Victoria, 3551 ...","[-36.76132, 144.2744]"
3,Endless Lifestyle Studio\r\n\r,Shop 3/188 Whitehorse Road\r\n\r,Balwyn,3103,23/05/2021 9:45am - 12:40pm,Case attended venue,26/05/2021,Tier 1 - Get tested immediately and quarantin...,Tier 1,"Shop 3/188 Whitehorse Road\r\n\r , Balwyn , ...","[-37.81264, 145.07602]"
4,Mcdonalds Bayswater North,296 Canterbury Road,Bayswater North,3153,21/05/2021 11:00pm - 12:00am,Case dined in restaurant,26/05/2021,Tier 1 - Get tested immediately and quarantin...,Tier 1,"296 Canterbury Road , Bayswater North , Vict...","[-37.83149, 145.27078]"


In [10]:
# Add lat, lng columns
sites_df['lat'] = sites_df.location.apply(lambda x: x[0])
sites_df['lng'] = sites_df.location.apply(lambda x: x[1])

### Create a GeoDataFrame

In [11]:
geometry = gpd.points_from_xy(sites_df.lng, sites_df.lat)
sites_gdf = gpd.GeoDataFrame(sites_df, geometry=geometry)

In [12]:
sites_gdf.head()

,site_name,address,suburb,postcode,period,note,date_added,advice,tier,full_address,location,lat,lng,geometry
0,Dukes Gym Abbotsford,571-573 Victoria Street,Abbotsford,3067,24/05/2021 6:15pm - 8:15pm,Case attended venue,26/05/2021,Tier 1 - Get tested immediately and quarantin...,Tier 1,"571-573 Victoria Street , Abbotsford , Victo...","[-37.81089, 145.00567]",-37.81089,145.00567,POINT (145.00567 -37.81089)
1,Chelsea Heights Hotel,Springvale Road & Wells Road,Aspendale Gardens,3196,23/05/2021 12:10pm - 1:30pm,Case dined at venue,27/05/2021,Tier 1 - Get tested immediately and quarantin...,Tier 1,"Springvale Road & Wells Road , Aspendale Gar...","[-38.02461, 145.1341]",-38.02461,145.13410,POINT (145.13410 -38.02461)
2,Axedale Tavern,105 High Street,Axedale,3551,23/05/2021 11:45am - 1:30pm,Case attended function,26/05/2021,Tier 1 - Get tested immediately and quarantin...,Tier 1,"105 High Street , Axedale , Victoria, 3551 ...","[-36.76132, 144.2744]",-36.76132,144.27440,POINT (144.27440 -36.76132)
3,Endless Lifestyle Studio\r\n\r,Shop 3/188 Whitehorse Road\r\n\r,Balwyn,3103,23/05/2021 9:45am - 12:40pm,Case attended venue,26/05/2021,Tier 1 - Get tested immediately and quarantin...,Tier 1,"Shop 3/188 Whitehorse Road\r\n\r , Balwyn , ...","[-37.81264, 145.07602]",-37.81264,145.07602,POINT (145.07602 -37.81264)
4,Mcdonalds Bayswater North,296 Canterbury Road,Bayswater North,3153,21/05/2021 11:00pm - 12:00am,Case dined in restaurant,26/05/2021,Tier 1 - Get tested immediately and quarantin...,Tier 1,"296 Canterbury Road , Bayswater North , Vict...","[-37.83149, 145.27078]",-37.83149,145.27078,POINT (145.27078 -37.83149)


In [13]:
# Save to geojson file
sites_gdf.drop(columns=["location"], inplace=True)
sites_gdf.to_file("../data/covid_sites.geojson", driver="GeoJSON")